# Capstone Project - Deduplication of Swissbib Raw Data

**Program** Applied Data Science : Machine Learning<br>
**Institution** EPFL Extension School<br>
**Course** \#5, Capstone Project<br><br>
**Title** Deduplication of Swissbib Raw Data<br>
**Author** Andreas Jud<br>
**Date** dd-mmm-2020

## Table of Contents

- [Introduction](#Introduction)
- [Overview](#overview)
- [Summary](#summary)

## Introduction<a id='introduction'/>

[Proposal](./project-proposal-andreas-jud.ipynb)

## Overview<a id='overview'/>

The notebook of the capstone project consists of the following chapters.

1. [Data Analysis](./1_DataAnalysis.ipynb)
- [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb)
- [Feature Matrix Generation](./3_FeatureMatrixGeneration.ipynb)
- [Decision Tree Model](/4_DecisionTreeModel.ipynb)

Appendix

A. [References](./A_References.ipynb)

## Summary<a id='summary'/>

In [1]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

# Determine all relenvant notebooks, ommit Overview Summary and Appendixes
a = ! ls [1-9]_* | grep .ipynb

for i in range(len(a)):
    print('Executing notebook', a[i])
    with open(a[i]) as notebook:
        nb = nbformat.read(notebook, as_version=4)
        ep = ExecutePreprocessor(timeout=None)
        ep.preprocess(nb, {"metadata": {"path": './'}})

print('Done with all notebooks.')

Executing notebook 1_DataAnalysis.ipynb
Executing notebook 2_GoldstandardDataPreparation.ipynb
Executing notebook 3_FeatureMatrixGeneration.ipynb
Executing notebook 4_FeatureDiscussionDummyBaseline.ipynb
Executing notebook 5_DecisionTreeModel.ipynb
Executing notebook 6_SVCModel.ipynb
Executing notebook 7_NeuralNetwork-Copy1.ipynb
Executing notebook 7_NeuralNetwork-Copy2.ipynb
Executing notebook 7_NeuralNetwork.ipynb
Done with all notebooks.


In [2]:
import results_saving_funcs as rsf
import pandas as pd

path_goldstandard = './daten_goldstandard'

df_results = rsf.restore_results(path_goldstandard)
df_results.reset_index(drop=True, inplace=True)

df_results.sort_values('test_score', ascending=False)

,model,test_score,auc,accuracy,precision,recall,test_score_log,auc_log,accuracy_log,precision_log,recall_log
1,DecisionTreeClassifier,0.999252,97.771539,99.925211,91.558442,95.593220,-7.198260,-3.803859,-7.198260,-2.472003,-3.122026
2,SVC,0.998102,86.085302,99.810152,92.608696,72.203390,-6.266702,-1.972225,-6.266702,-2.604866,-1.280256
0,DummyClassifier,0.988973,49.898528,98.897348,0.354610,0.338983,-4.507452,-0.691120,-4.507452,-0.003552,-0.003396
3,Neural Network,0.000000,96.610169,96.610169,95.379538,97.966102,0.000000,-3.384390,-3.384390,-3.074675,-3.895216
4,Neural Network,0.000000,97.842220,97.474168,93.890675,98.983051,0.000000,-3.836090,-3.678600,-2.795354,-4.588363
5,Neural Network,0.000000,96.949153,96.949153,96.949153,96.949153,0.000000,-3.489751,-3.489751,-3.489751,-3.489751
